In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from shapely.geometry import Point
import shapely
pd.set_option('display.max_columns', None)


# Load Data

In [ ]:
buildings_df = gpd.read_file('../data/Buildings/')
parcels_df = gpd.read_file('../data/Parcels_shape/')

# Find buildings in parcels

In [ ]:
buildings_df_temp = buildings_df.copy()

In [ ]:
buildings_df_temp.geometry = buildings_df_temp.geometry.centroid
buildings_df_temp = buildings_df_temp[buildings_df_temp.is_valid]
parcels_df = parcels_df[parcels_df.is_valid]

In [ ]:
building_to_parcels = gpd.sjoin(
    buildings_df_temp[['OBJECTID','AddNum','Street','geometry']], 
    parcels_df[['MBL','geometry','AddNum','Street']], 
    how = 'inner', op = 'within'
)


In [ ]:
buildings_parcels_df = (
    buildings_df
    .merge(building_to_parcels[['OBJECTID','MBL']])
    .merge(parcels_df, on = 'MBL')[['MBL', 'geometry_x', 'geometry_y','AddNum_y','Street_y']]
    .rename({'geometry_x':'building', 'geometry_y':'parcel'}, axis = 1)
)

In [ ]:
buildings_parcels_df = buildings_parcels_df[buildings_parcels_df.parcel.apply(type) == shapely.geometry.polygon.Polygon]

In [ ]:
buildings_parcels_df

# Create Features


1. distance to each edge
2. buildings per parcel
3. number of edges in building footprint

### distance to each edge

In [ ]:
row = buildings_parcels_df.loc[0]

In [ ]:
def get_parcel_edge_midpoints(parcel):
    # simplify parcel so their are 4 coordinates
    tolerance_low = 0
    tolerance_high = 1e10
    num_coord = 0
        
    while True:
        tolerance = (tolerance_low + tolerance_high)/2
        parcel_simple = parcel.simplify(tolerance)
        try:
            parcel_corners = parcel_simple.boundary.coords
        except:
            return np.NaN
        num_coord = len(parcel_corners)
        if num_coord == 5 or tolerance_high - tolerance_low < 1e-10:
            break
        elif num_coord < 5:
            tolerance_high = tolerance
        else:
            tolerance_low = tolerance
    
    if num_coord == 4:
        parcel_corners = list(parcel_corners)
        parcel_corners.append(parcel_corners[1])

    # get midpoints of parcel edges
    midpoints = []
    for i in range(4):
        corner1 = np.array(parcel_corners[i])
        corner2 = np.array(parcel_corners[i+1])
        midpoints.append(Point((corner1 + corner2)/2))

    # get orientation
    if midpoints[0].distance(midpoints[2]) > midpoints[1].distance(midpoints[3]):
        midpoint_dict = {
            'side1': midpoints[1],
            'side2': midpoints[3],
            'front1': midpoints[0],
            'front2': midpoints[2]
        }
    else:
         midpoint_dict = {
            'side1': midpoints[0],
            'side2': midpoints[2],
            'front1': midpoints[1],
            'front2': midpoints[3]
        }
    return midpoint_dict

In [ ]:
buildings_parcels_df['midpoints'] = buildings_parcels_df.parcel.apply(get_parcel_edge_midpoints)

buildings_parcels_df = buildings_parcels_df[~buildings_parcels_df.midpoints.isna()]

In [ ]:
for clm in ['side1','side2','front1','front2']:
    buildings_parcels_df[clm + '_dist'] = buildings_parcels_df.apply(
        lambda row: row.building.distance(row.midpoints[clm]), axis = 1)

In [ ]:
buildings_parcels_df['side_diff'] = np.absolute(buildings_parcels_df.side1_dist - buildings_parcels_df.side2_dist)
buildings_parcels_df['front_diff'] = np.absolute(buildings_parcels_df.front1_dist - buildings_parcels_df.front2_dist)

In [ ]:
buildings_parcels_df

### buildings per parcel

In [ ]:
building_per_parcels = gpd.sjoin(
    buildings_df_temp[['OBJECTID','AddNum','Street','geometry']], 
    parcels_df[['MBL','geometry','AddNum','Street']], 
    how = 'right', op = 'within'
)

In [ ]:
building_per_parcels = building_per_parcels.groupby('MBL').size().reset_index().rename({0:'building_count'}, axis=1)

In [ ]:
buildings_parcels_df = buildings_parcels_df.merge(building_per_parcels, how = 'left')

In [ ]:
buildings_parcels_df

## number of edges in building footprint

In [ ]:
def get_edge_count(polygon):
    try:
        return len(polygon.boundary.coords)
    except NotImplementedError:
        return np.NaN
buildings_parcels_df['building_edges'] = buildings_parcels_df.building.apply(get_edge_count)
buildings_parcels_df['parcel_edges'] = buildings_parcels_df.parcel.apply(get_edge_count)

In [ ]:
buildings_parcels_df

## save

In [ ]:
buildings_parcels_df = buildings_parcels_df[[
    'MBL',
    'side1_dist',
    'side2_dist',
    'front1_dist',
    'front2_dist',
    'side_diff',
    'front_diff',
    'building_count',
    'building_edges',
    'parcel_edges'
]]


In [ ]:
buildings_parcels_df.groupby('MBL').min()

In [ ]:
buildings_parcels_df.to_csv('../data/building_parcel_geometric_features.csv')

## Figures for Poster

In [ ]:
from matplotlib import pyplot as plt
from shapely.geometry.polygon import Polygon
from descartes import PolygonPatch

In [ ]:
row = buildings_parcels_df[buildings_parcels_df.MBL == '73-B-6'].iloc[0]
row

In [ ]:
GREY = '#A8A8A8'
RED = '#AC2A25'
pavlos = row.building
pavlos_parcel = row.parcel

fig = plt.figure(dpi = 450) 
ax = fig.gca() 
ax.add_patch(PolygonPatch(pavlos_parcel, fc=GREY, ec=GREY, alpha=1, zorder=2 ))
ax.add_patch(PolygonPatch(pavlos, fc=RED, ec=RED, alpha=1, zorder=2 ))

ax.axis('scaled')
plt.savefig('../figures/pavlos.jpg')

In [ ]:
pavlos = row.building
pavlos_parcel = row.parcel.simplify(100)

fig = plt.figure(dpi = 450) 
ax = fig.gca() 
ax.add_patch(PolygonPatch(pavlos_parcel, fc=GREY, ec=GREY, alpha=1, zorder=2))
ax.add_patch(PolygonPatch(pavlos, fc=RED, ec=RED, alpha=1, zorder=2 ))

ax.axis('scaled')
plt.savefig('../figures/pavlos_simple.jpg')